# SETUP

## is GPU on?

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## boilerplate

In [1]:
from tensorflow.keras.callbacks import Callback, EarlyStopping
from numpy import arange

MODEL_NAME = 'inception3'
IMG_SIZE = (299, 299)
INPUT_SHAPE=(299, 299, 3)
CLASSES = 2
TRIAL = list(arange(0,3))
FT_BLOCK = 10 # FROM feature extractor TO fine tuning scratch
BATCH_SIZE = 64 # these data points will be passed as a batch at one time to the network
PR_RATIO = list(arange(1,10,1))
TRIAL = list(arange(0,10,1))
MODEL_DIR = '../../models/PR'

## UDC/Fs

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from math import floor
from timeit import default_timer as timer
from json import dump
from tensorflow.keras.models import Model

def model_constructor(FT_BLOCK):
    base_model = InceptionV3(
        weights='imagenet', 
        include_top=False,
        input_shape=INPUT_SHAPE)
    
    # construct the top layer containing 
    x = base_model.output
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x) # add a global spatial average pooling layer
    x = layers.Dense(CLASSES, activation='sigmoid',  name='predictions')(x) # and add the output layer for binary class

    # model is ready to be trained
    model = Model(inputs=base_model.input, outputs=x)

    # freeze the layers before the `FROZEN_END` layer 
    total_layers = len(base_model.layers)
    ft_layers = floor(total_layers/10)
    FROZEN_END = total_layers - ft_layers * FT_BLOCK

    base_model.trainable = True
    for layer in base_model.layers[:FROZEN_END]: 
        layer.trainable = False
    
    return model

class TimeCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def time_converter(sec):
    hours, rem = divmod(sec, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

def save_history(history, tag):
    file_path = f'{HISTORY_DIR}/{tag}.json'
    with open(file_path, 'w') as f:
        dump(history.history, f)
        
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import data
def data_preparation(BATCH_SIZE):
    train_dir = '../../data/binaryclass_clean/train/'
    test_dir = '../../data/binaryclass_clean/test/'

    train_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='training')

    val_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='validation')

    test_ds = image_dataset_from_directory(
        directory=test_dir,
        label_mode='categorical',
        batch_size=1,
        image_size=IMG_SIZE)

    AUTOTUNE = data.AUTOTUNE
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

import tempfile
from os.path import getsize

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in Mbytes.
  import os
  import zipfile

  zipped_file = f'{MODEL_DIR}/{tag}.zip'
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
    zipped_size = getsize(zipped_file) / float(2**20)
#     remove(zipped_file)

  return zipped_size

## load data

In [3]:
train_ds, val_ds, test_ds = data_preparation(BATCH_SIZE)

Found 9980 files belonging to 2 classes.
Using 8982 files for training.
Found 9980 files belonging to 2 classes.
Using 998 files for validation.
Found 1000 files belonging to 2 classes.


# GRID SEARCH

In [4]:
# !pip install --user --upgrade tensorflow-model-optimization

In [4]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import save_model

for i in TRIAL:
    print(" - - - - - TRIAL:", i, " - - - - - ")
    loss, accuracy = [None]*len(PR_RATIO), [None]*len(PR_RATIO)

    for pr in PR_RATIO:
        tag = f'{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}_PR{pr*10}_{i}'
        print(tag)

        logdir = f'../../logs/PR/{tag}.json'
        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(
                target_sparsity=pr/10, begin_step=0, end_step=-1, frequency=1)}

        model = model_constructor(FT_BLOCK)
        prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
        model_for_pruning = prune_low_magnitude(model, **pruning_params)
        # model_for_pruning.summary()

        model_for_pruning.compile(loss='binary_crossentropy', metrics='accuracy', optimizer = 'adam')
        history = model_for_pruning.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=callbacks)

        loss[pr-1], accuracy[pr-1] = model_for_pruning.evaluate(test_ds)

    # PRINT THE RESULT
    print(f' * * * * * {tag[:-2]}10-90_LOSS * * * * * ')
    for item in loss:
        print(item)
    print(f' * * * * * {tag[:-2]}10-90_ACCURACY * * * * * ')
    for item in accuracy:
        print(item)

 - - - - - TRIAL: 0  - - - - - 
inception3_BS64_FT100_PR10_0


/usr/local/lib/python3.6/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


1000/1000 [==============================] - 35s 35ms/step - loss: 0.6252 - accuracy: 0.9240
inception3_BS64_FT100_PR20_0
1000/1000 [==============================] - 36s 36ms/step - loss: 0.5026 - accuracy: 0.9210
inception3_BS64_FT100_PR30_0
1000/1000 [==============================] - 35s 35ms/step - loss: 0.7687 - accuracy: 0.9050
inception3_BS64_FT100_PR40_0
1000/1000 [==============================] - 36s 36ms/step - loss: 0.5824 - accuracy: 0.9280
inception3_BS64_FT100_PR50_0
1000/1000 [==============================] - 33s 33ms/step - loss: 0.6031 - accuracy: 0.9280
inception3_BS64_FT100_PR60_0
1000/1000 [==============================] - 36s 36ms/step - loss: 0.5884 - accuracy: 0.9330
inception3_BS64_FT100_PR70_0
1000/1000 [==============================] - 36s 36ms/step - loss: 0.6606 - accuracy: 0.9090
inception3_BS64_FT100_PR80_0
1000/1000 [==============================] - 36s 36ms/step - loss: 1.1121 - accuracy: 0.8130
inception3_BS64_FT100_PR90_0
1000/1000 [=============

1000/1000 [==============================] - 28s 28ms/step - loss: 0.6749 - accuracy: 0.9110
inception3_BS64_FT100_PR90_2
1000/1000 [==============================] - 27s 27ms/step - loss: 0.7048 - accuracy: 0.9140
 * * * * * inception3_BS64_FT100_PR9010-90_LOSS * * * * * 
0.3352678418159485
0.6172307133674622
0.5860668420791626
0.5713390707969666
0.4830091893672943
0.491891473531723
0.6997840404510498
0.6748652458190918
0.7048263549804688
 * * * * * inception3_BS64_FT100_PR9010-90_ACCURACY * * * * * 
0.9340000152587891
0.925000011920929
0.9380000233650208
0.9319999814033508
0.9309999942779541
0.9309999942779541
0.9039999842643738
0.9110000133514404
0.9139999747276306
 - - - - - TRIAL: 3  - - - - - 
inception3_BS64_FT100_PR10_3
1000/1000 [==============================] - 27s 27ms/step - loss: 0.5498 - accuracy: 0.9260
inception3_BS64_FT100_PR20_3
1000/1000 [==============================] - 28s 28ms/step - loss: 0.5328 - accuracy: 0.9190
inception3_BS64_FT100_PR30_3
1000/1000 [=======

1000/1000 [==============================] - 27s 27ms/step - loss: 0.7195 - accuracy: 0.9170
 * * * * * inception3_BS64_FT100_PR9010-90_LOSS * * * * * 
1.3492212295532227
0.5744739770889282
0.6092304587364197
0.6299102306365967
0.4791872501373291
0.5139113068580627
0.6973498463630676
0.5153729319572449
0.7194779515266418
 * * * * * inception3_BS64_FT100_PR9010-90_ACCURACY * * * * * 
0.890999972820282
0.9169999957084656
0.8960000276565552
0.9100000262260437
0.9139999747276306
0.9190000295639038
0.9150000214576721
0.9319999814033508
0.9169999957084656
 - - - - - TRIAL: 8  - - - - - 
inception3_BS64_FT100_PR10_8
1000/1000 [==============================] - 27s 27ms/step - loss: 0.5745 - accuracy: 0.9200
inception3_BS64_FT100_PR20_8
1000/1000 [==============================] - 28s 27ms/step - loss: 0.8294 - accuracy: 0.8960
inception3_BS64_FT100_PR30_8
1000/1000 [==============================] - 27s 27ms/step - loss: 0.7852 - accuracy: 0.9150
inception3_BS64_FT100_PR40_8
1000/1000 [======

# MODEL WITHOUT PRUNE

In [5]:
model = model_constructor(FT_BLOCK)
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
model.fit(train_ds, epochs=100, verbose=1, validation_data=val_ds)
# Save the model
model_path = f'{MODEL_DIR}/{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}.h5'
model.save(model_path)

Epoch 1/100
141/141 [==============================] - 62s 361ms/step - loss: 0.2216 - accuracy: 0.9125 - val_loss: 0.4667 - val_accuracy: 0.9038
Epoch 2/100
141/141 [==============================] - 52s 353ms/step - loss: 0.1400 - accuracy: 0.9432 - val_loss: 0.2743 - val_accuracy: 0.9429
Epoch 3/100
141/141 [==============================] - 51s 350ms/step - loss: 0.1268 - accuracy: 0.9500 - val_loss: 0.4027 - val_accuracy: 0.9108
Epoch 4/100
141/141 [==============================] - 51s 351ms/step - loss: 0.1093 - accuracy: 0.9547 - val_loss: 0.2010 - val_accuracy: 0.9419
Epoch 5/100
141/141 [==============================] - 51s 352ms/step - loss: 0.1055 - accuracy: 0.9525 - val_loss: 0.2130 - val_accuracy: 0.9519
Epoch 6/100
141/141 [==============================] - 51s 348ms/step - loss: 0.0935 - accuracy: 0.9569 - val_loss: 0.3218 - val_accuracy: 0.9379
Epoch 7/100
141/141 [==============================] - 51s 351ms/step - loss: 0.0848 - accuracy: 0.9597 - val_loss: 0.1261 -

Epoch 57/100
141/141 [==============================] - 52s 353ms/step - loss: 0.0305 - accuracy: 0.9817 - val_loss: 0.1924 - val_accuracy: 0.9539
Epoch 58/100
141/141 [==============================] - 51s 349ms/step - loss: 0.0306 - accuracy: 0.9824 - val_loss: 0.2538 - val_accuracy: 0.9529
Epoch 59/100
141/141 [==============================] - 51s 351ms/step - loss: 0.0419 - accuracy: 0.9776 - val_loss: 0.3116 - val_accuracy: 0.9549
Epoch 60/100
141/141 [==============================] - 51s 351ms/step - loss: 0.0343 - accuracy: 0.9803 - val_loss: 0.2437 - val_accuracy: 0.9409
Epoch 61/100
141/141 [==============================] - 51s 348ms/step - loss: 0.0270 - accuracy: 0.9839 - val_loss: 0.2228 - val_accuracy: 0.9459
Epoch 62/100
141/141 [==============================] - 50s 340ms/step - loss: 0.0321 - accuracy: 0.9811 - val_loss: 0.2507 - val_accuracy: 0.9539
Epoch 63/100
141/141 [==============================] - 52s 353ms/step - loss: 0.0263 - accuracy: 0.9846 - val_loss: 0

In [6]:
model.evaluate(test_ds)

1000/1000 [==============================] - 29s 29ms/step - loss: 0.5672 - accuracy: 0.9240


[0.5672118067741394, 0.9240000247955322]

In [7]:
print("Model size in Mb:", getsize(model_path) / float(2**20))

Model size in Mb: 250.6614761352539
